In [1]:
import pandas as pd
from keras.models import Sequential, Model
import keras.layers as layers
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import random
import os
import pdb
import matplotlib.pyplot as plt
from copy import deepcopy
import tensorflow as tf
import time
from tensorflow.python.client import device_lib
import sys
import keras
 
print("TF INFO:", device_lib.list_local_devices())
print('GPU INFO:', K.tensorflow_backend._get_available_gpus())

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
K.set_session(tf.Session(config=config))
K.tensorflow_backend.set_session(tf.Session(config=config))


Using TensorFlow backend.
/Users/fahmid/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/fahmid/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/fahmid/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/fahmid/anaconda3/lib/python3.7/site-packages/t

TF INFO: [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6510251721243927723
]
GPU INFO: []


# DQN Algorithm

In [2]:
class DQNAgent:
    def __init__(self, df_batch, state_size, action_size,
                 minibatch_size=128, gamma=.95, lr=0.001, units=128, priority_aplha = 0.5, 
                 lookback=3, is_lstm=False, layers=4, bcq=0,
                 copy_online_to_target_ep=100, eval_after=100, mode="normal"):
        """
        creates a DQN Agent for batch learning
        param: df_batch is the batch data in MDP format
        param: state_size
        param: action_size
        param: minibatch_size 
        param: gamma
        param: lr
        param: units
        param: priority_aplha for Prioritized Experience Reply. 0 Makes it Vanilla Experience Reply
        param: copy_online_to_target_ep copies current network to terget network. meaningless for Double DQN
        param: eval_after 
        param: lookback how many historical states are inluced including the current one
        param: is_lstm is for layer type
        param: layers is for number of layers including output layer
        param: mode is for baselines. normal is no baseline. other options are: random, 0, 1, 2, 3
        
        """
        
        #adding priority as noise in batch
        df_batch.at[:, 'weight'] = 0.0
        for i, row in df_batch.iterrows():
            df_batch.at[i, 'priority'] = (np.random.uniform(0, 0.001))**priority_aplha
        
        # setting parameters
        self.state_size = state_size
        self.action_size = action_size
        self.batch = df_batch
        
        self.minibatch_size = minibatch_size
        self.gamma = gamma
        self.learning_rate = lr
        self.units = units
        self.priority_aplha = priority_aplha
        self.lookback = lookback
        self.is_lstm = is_lstm
        self.layers = layers
        if self.layers<3:
            print("MIN LAYERS SHOULD BE 3. FORCING 3 LAYERS (including output)")
            self.layers = 3
        self.bcq = bcq
        
        self.copy_online_to_target_ep = copy_online_to_target_ep
        self.eval_after = eval_after
        
        self.batch = self._setup_lookback_states(self.batch)
        
        self.mode = mode
        
        # setting up the models
        self.model_1 = self._build_model()
        self.model_2 = self._build_model()
        
        
        # evaluation variables
        self.ecrs = []
        self.IS = []
        self.WIS = []
        self.PDIS = []
        self.PDWIS = []
        self.DR = []
        self.remediations = []
    
    
    def _setup_lookback_states(self, df):
        curr_ep = -1
        for i, row in df.iterrows():
            if curr_ep!=row['episode_id']:
                curr_ep = row['episode_id']
                prevs = []
                for j in range(self.lookback-1):
                    prevs.append(np.full(shape=self.state_size, fill_value=0))
                state = row['state'] 
                prevs.append(state)
                pervs = deepcopy(prevs)
            df.at[i, 'state'] = np.array(prevs)
            prevs = deepcopy(prevs[1:])
            prevs.append(row['next_state'])
            pervs = deepcopy(prevs)
            df.at[i, 'next_state'] = np.array(prevs)
            prevs = deepcopy(prevs)    
        
        for i, row in df.iterrows():
            state, next_state = self.get_transformed_state(row)
            df.at[i, 'state'] = state
            df.at[i, 'next_state'] = next_state
        
        
        return df
    
    def get_transformed_state(self, row):
        if self.is_lstm:
            state = row['state'].reshape(1, self.lookback, self.state_size)
            next_state = row['next_state'].reshape(1, self.lookback, self.state_size)
        else:
            state = row['state'].reshape(1, self.state_size * self.lookback)
            next_state = row['next_state'].reshape(1, self.state_size * self.lookback)
        
        return state, next_state
    
    def _build_model(self):
        """
        Standard DQN model
        """
        model = Sequential()
        
        if self.is_lstm:
            # 1 layer
            model.add(layers.LSTM(self.units, input_shape=(self.lookback, self.state_size), 
                                  activation='relu', kernel_regularizer=keras.regularizers.l2(), 
                                  return_sequences=True, kernel_initializer='glorot_normal'))
            
            for i in range(self.layers-3):
                model.add(layers.LSTM(self.units, activation='relu', kernel_regularizer=keras.regularizers.l2(), 
                                      return_sequences=True, kernel_initializer='glorot_normal'))
            # 1 layer
            model.add(layers.LSTM(self.units, activation='relu', kernel_regularizer=keras.regularizers.l2(), 
                                  return_sequences=False, kernel_initializer='glorot_normal'))
        else:
            model.add(layers.Dense(self.units, input_dim=self.state_size * self.lookback, activation='relu', 
                                   kernel_regularizer=keras.regularizers.l2(), kernel_initializer='glorot_normal'))
            for i in range(self.layers-2):
                model.add(layers.Dense(self.units, activation='relu', 
                                       kernel_regularizer=keras.regularizers.l2(), kernel_initializer='glorot_normal'))
            
        # 1 layer
        model.add(layers.Dense(self.action_size, activation='linear', 
                               kernel_regularizer=keras.regularizers.l2(), kernel_initializer='glorot_normal'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate), metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mae'])
        return model
    
    def act(self, state):
        act_values = self.model_2.predict(state)
        return np.argmax(act_values[0]), np.max(act_values[0])
    
    def state_value(self, state):
        act_values = self.model_2.predict(state)
        return np.sum(act_values[0]) 
    
    def q_value(self, state, action):
        act_values = self.model_2.predict(state)
        return act_values[0][action] 
    
    def _filter_bcq(self, row, ns_act_values):
        if self.bcq==0:
            return np.argmax(ns_act_values)
        
        gw = self.batch.loc[self.batch['cluster']==row['ns_cluster']].sample(100)
        gwa = gw.groupby(['action']).count()['episode_id'].tolist()
        gwap = np.array(gwa)/100
        
        for i, p in enumerate(gwap):
            if p<self.bcq:
                ns_act_values[i] = -9999
        
        return np.argmax(ns_act_values)
    
    def _fit_model(self, row):
        i = row.name
        state, action, reward, next_state, done = row['state'], row['action'], row['reward'], row['next_state'], row['done']
            
        target_q = reward
        
        if not done:    
            if self.mode=="normal":
                ns_act_values = self.model_1.predict(next_state)[0]
#                 a_prime = np.argmax(ns_act_values)
                a_prime = self._filter_bcq(row, ns_act_values)
            elif self.mode=="random":
                a_prime = np.random.choice(range(self.action_size))
            else:
                a_prime = int(self.mode)
            
            target_ns_act_values = self.model_2.predict(next_state)[0]
            target_ns_q = target_ns_act_values[a_prime]
                           

            target_q = reward + self.gamma*target_ns_q

            self.batch.loc[i, 'pred_action'] = a_prime
            self.batch.loc[i, 'pred_q'] = target_q
        
        target_f = self.model_1.predict(state)

        # Prioritized Experience Reply with noise
        
        self.batch.loc[i, 'priority'] = (abs(target_q - target_f[0][action]) + np.random.uniform(0, 0.001))**self.priority_aplha

        target_f[0][action] = target_q
        self.model_1.fit(state, target_f, epochs=1, verbose=0)
        
    
    def _learn_minibatch(self):
        priority_sum = self.batch['priority'].sum()
        self.batch['weight'] = self.batch['priority']/priority_sum
        minibatch = self.batch.sample(self.minibatch_size, weights=self.batch['weight'])
        minibatch.apply(self._fit_model, axis=1) 
            
    
    def ecr_reward(self):
        reward = 0.0
        count = 0
        for i, row in self.batch.loc[self.batch['transition_id']==0].iterrows():
            state = row['state']
            reward += self.act(state)[1]
            count += 1
            
        ecr = reward/count
        self.ecrs.append(ecr)
        return ecr

    
    def get_ips(self, action):
        if self.action_size==2:
            ips = 1.0/0.5
        else:
            if action == 0 or action == 1:
                ips = 1.0/0.1
            else:
                ips = 1.0/0.4
        return ips
        
    
    
    def get_eval(self):
        # set up roh with action_predicted
        self.batch['roh'] = -1.0
        curr_ep = -1
        for i, row in self.batch.iterrows():
            if row['episode_id']!=curr_ep:
                roh_t = 1
                curr_ep = row['episode_id']
            if row['action']!=row['pred_action']:
                ips = 0
            else:
                ips = self.get_ips(row['action'])
                
            roh_t *= ips
            self.batch.at[i, 'roh'] = roh_t
        
        total_eps = len(self.batch['episode_id'].unique())
        # equation found in emma brunskill's lecture note
        
        # as each roh is calculated multiplicatively, the last roh is the entire multiplication result
        # summing (gamma**t)*(R_t^i) will result to delayed reward
        # we can take the last roh (where done is true) and the delayed reward for each ep
        a_is = sum(self.batch.loc[self.batch['done']==True].apply(lambda x: x['roh'] * x['delayed_reward'], axis=1))
        isamp = a_is/total_eps
        
        # for weighted important samplng
        sum_roh = sum(self.batch.loc[self.batch['done']==True, 'roh'])
        if sum_roh==0:
            wis = 0
        else:
            wis = a_is/sum_roh
        
        pdis = 0
        for transition_id in self.batch['transition_id'].unique():
            d = self.batch.loc[self.batch['transition_id']==transition_id]
            a_pdis = (self.gamma**transition_id) * sum(d.apply(lambda x: x['roh'] * x['reward'], axis=1))
            pdis += (a_pdis/total_eps)
        
        curr_ep = -1
        trans = -1
        dr = 0
        pdwis = 0
        pdwis_nom = 0
        pdwis_denom = 0
        for i, row in self.batch.iterrows():
            if curr_ep!=row['episode_id']:
                curr_ep = row['episode_id']
                trans = 0
            if row['transition_id']!=trans:
                print("ERROR:", curr_ep, row['transition_id'], trans)
            
            gamma_t = self.gamma**row['transition_id']
            q_pi_e = self.q_value(row['state'], row['action'])
            v_pi_e = self.state_value(row['state'])
            roh_t_i = row['roh']
            if trans!=0:
                roh_t_sub_1_i = self.batch.loc[(self.batch['episode_id']==row['episode_id']) &
                                                  (self.batch['transition_id']==(trans-1)), 'roh'].tolist()[0]
            else:
                roh_t_sub_1_i = 1
            
            dr += ((gamma_t*roh_t_i*(row['reward'] - q_pi_e)) + (gamma_t*roh_t_sub_1_i*v_pi_e))
            
            # PDWIS
            sum_roh_t = float(sum(self.batch.loc[self.batch['transition_id']==row['transition_id']]['roh']))
            if sum_roh_t==0:
                w_t_i = 0
            else:
                w_t_i = row['roh']/sum_roh_t
            pdwis_nom += (w_t_i * gamma_t * row['reward'])
            pdwis_denom += (w_t_i * gamma_t)
            
            trans += 1
        
        dr = dr/total_eps
        pdwis = pdwis_nom/pdwis_denom
        
        
        self.IS.append(isamp)
        self.WIS.append(wis)
        self.PDIS.append(pdis)
        self.PDWIS.append(pdwis)
        self.DR.append(dr)
        
        return isamp, wis, pdis, pdwis, dr
        
    
    def predict(self):
        self.batch['pred_action'] = -1
        self.batch['pred_q'] = 0
        self.batch.apply(self._predict_row, axis=1)
    
    def _predict_row(self, row):
        i = row.name
        state = row['state']
        act, q = self.act(state)
        self.batch.loc[i, 'pred_action'] = act
        self.batch.loc[i, 'pred_q'] = q
    
    def learn(self, epoch):
        for i in range(epoch):
            self._learn_minibatch()
            
            if (i+1)%self.copy_online_to_target_ep==0:
                self.model_2.set_weights(self.model_1.get_weights())
            
            if (i+1)%self.eval_after==0:
                t1 = time.time()
                self.predict()
                
                ecr = self.ecr_reward()
                isamp, wis, pdis, pdwis, dr = self.get_eval()
                t2 = time.time()
                print("Eval Time:", (t2-t1))
                print("--epoch: {}/{} | ECR: {:.5f} | IS: {:.5f} | WIS: {:.5f} | PDIS: {:.5f} | PDWIS: {:.5f} | DR: {:.5f} --".format(i+1, epoch, ecr, isamp, wis, pdis, pdwis, dr))
                self.summary()
        
        self.model_2.set_weights(self.model_1.get_weights())
        self.predict()
                
    
    def get_all_eval_df(self):
        eval_df = pd.DataFrame(columns=['ECR', 'IS', 'WIS', 'PDIS', 'PDWIS', 'DR', 'REMEDIATION'])
        
        eval_df['ECR'] = self.ecrs
        eval_df['IS'] = self.IS
        eval_df['WIS'] = self.WIS
        eval_df['PDIS'] = self.PDIS
        eval_df['PDWIS'] = self.PDWIS
        eval_df['DR'] = self.DR
        eval_df['REMEDIATION'] = self.remediations
        
        return eval_df
    
    
    def summary(self):
        pred_const = len(self.batch.loc[self.batch['pred_action'] == 3]) 
        pred_active = len(self.batch.loc[self.batch['pred_action'] == 2])
        pred_pass = len(self.batch.loc[self.batch['pred_action'] == 1])
        pred_none = len(self.batch.loc[self.batch['pred_action'] == 0])

        self.remediations.append({"constructive": pred_const, "active": pred_active, "passive": pred_pass,
                                "none": pred_none})
        print("Pred-> Constructive: {}, Active: {}, Passive: {}, None: {}"
              .format(pred_const, pred_active, pred_pass, pred_none))
        
    
    

# Result

In [3]:
def summary_result(df):

    true_const = len(df.loc[df['action'] == 3])
    true_active = len(df.loc[df['action'] == 2])
    true_pass = len(df.loc[df['action'] == 1])
    true_none = len(df.loc[df['action'] == 0])
    
    pred_const = len(df.loc[df['pred_action'] == 3]) 
    pred_active = len(df.loc[df['pred_action'] == 2])
    pred_pass = len(df.loc[df['pred_action'] == 1])
    pred_none = len(df.loc[df['pred_action'] == 0])
     
    
    print("True-> Constructive: {}, Active: {}, Passive: {}, None: {}"
          .format(true_const, true_active, true_pass, true_none))
    print("Pred-> Constructive: {}, Active: {}, Passive: {}, None: {}"
          .format(pred_const, pred_active, pred_pass, pred_none))
    
    
    
    true_reward = df.loc[df['done']==True]['reward']
    pred_reward = df.loc[df['transition_id']==0]['pred_q']
    true_reward_mean = np.mean(true_reward)
    true_reward_std = np.std(true_reward)
    pred_reward_mean = np.mean(pred_reward)
    pred_reward_std = np.std(pred_reward)
    
    print("-> True Reward: {:.5f}/{:.5f}, Pred Reward: {:.5f}/{:.5f}".format(
        true_reward_mean, true_reward_std, pred_reward_mean, pred_reward_std))

    
    ret_dict = {"True": {"Constructive": true_const, "Active": true_active, "Passive": true_pass, 
                         "None": true_none, "RewardMean": true_reward_mean, "RewardStd": true_reward_std},
               "Pred": {"Constructive": pred_const, "Active": pred_active, "Passive": pred_pass, 
                         "None": pred_none, "RewardMean": pred_reward_mean, "RewardStd": pred_reward_std}
               }
    
    
    
    return ret_dict

# Run Program

In [6]:
df_org = pd.read_pickle('../temp/df_all_norm_cluster.pkl')
df = df_org.copy()
epoch = 20000
df['reward'] = df['delayed_reward']
result_dir = '../result/'
def run(agent, run_name, epoch):
    print("===== STARTING ", run_name, "=====")
    agent.learn(epoch)
    result_df = agent.batch
    eval_df = agent.get_all_eval_df()
    eval_df.to_pickle(result_dir + run_name +'_eval.pkl')
    summary_result(result_df)
    result_df.to_pickle(result_dir + run_name +'_result.pkl')

    return result_df, eval_df

for is_lstm in [True]:
    for lookback in [1]:
        for random_state in [1,2,3,4,5]:
            np.random.seed(random_state)
            random.seed(random_state)
            tf.set_random_seed(random_state)
            print("TF INFO:", device_lib.list_local_devices())
            print('GPU INFO:', K.tensorflow_backend._get_available_gpus())

            config = tf.ConfigProto(log_device_placement=True)
            config.gpu_options.per_process_gpu_memory_fraction = 0.9
            K.set_session(tf.Session(config=config))
            K.tensorflow_backend.set_session(tf.Session(config=config))

            
            run_name = "DQN_is_lstm_"+ str(is_lstm) + "_lookback_" + str(lookback) + "_run_" + str(epoch) + "_rs_" + str(random_state)
            agent = DQNAgent(df_batch=df.copy(), state_size=len(df.iloc[0]['state']), action_size=4, 
                             copy_online_to_target_ep=100, eval_after=100, bcq=0.1,
                             lookback=lookback, layers=4, is_lstm=is_lstm, mode="normal")
            result_df, eval_df = run(agent, run_name, epoch)



TF INFO: [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1448305049376309946
]
GPU INFO: []
===== STARTING  DQN_is_lstm_True_lookback_1_run_20000_rs_1 =====
Eval Time: 23.228199243545532
--epoch: 100/20000 | ECR: 5.71830 | IS: 1.62567 | WIS: 63.33600 | PDIS: 1.62567 | PDWIS: 0.52367 | DR: 54.08142 --
Pred-> Constructive: 795, Active: 0, Passive: 1341, None: 2862
Eval Time: 23.105072259902954
--epoch: 200/20000 | ECR: 13.42545 | IS: 1.36899 | WIS: 66.67000 | PDIS: 1.36899 | PDWIS: 0.51939 | DR: 147.72490 --
Pred-> Constructive: 0, Active: 0, Passive: 1571, None: 3427
Eval Time: 23.135251998901367
--epoch: 300/20000 | ECR: 16.08883 | IS: 11.54320 | WIS: 63.34127 | PDIS: 10.96524 | PDWIS: 8.09803 | DR: 186.97350 --
Pred-> Constructive: 2029, Active: 0, Passive: 2969, None: 0
Eval Time: 23.255128145217896
--epoch: 400/20000 | ECR: 23.15277 | IS: 13.94529 | WIS: 60.70486 | PDIS: 13.19203 | PDWIS: 4.91045 | DR: 278.71897 --
Pred-> Constructive: 182

Eval Time: 23.495348930358887
--epoch: 3900/20000 | ECR: 44.86792 | IS: 6.79936 | WIS: 53.51576 | PDIS: 6.52196 | PDWIS: 25.51824 | DR: 357.53618 --
Pred-> Constructive: 697, Active: 0, Passive: 4173, None: 128
Eval Time: 23.504768133163452
--epoch: 4000/20000 | ECR: 45.42400 | IS: 25.08642 | WIS: 64.19486 | PDIS: 23.05495 | PDWIS: 7.75403 | DR: 592.49287 --
Pred-> Constructive: 2511, Active: 0, Passive: 2130, None: 357
Eval Time: 23.716893911361694
--epoch: 4100/20000 | ECR: 46.62831 | IS: 15.12631 | WIS: 66.96830 | PDIS: 14.14867 | PDWIS: 3.05442 | DR: 780.22356 --
Pred-> Constructive: 0, Active: 4685, Passive: 106, None: 207
Eval Time: 25.172689199447632
--epoch: 4200/20000 | ECR: 43.18216 | IS: 37.16895 | WIS: 68.67112 | PDIS: 32.94580 | PDWIS: 10.70768 | DR: 580.60529 --
Pred-> Constructive: 2088, Active: 1721, Passive: 0, None: 1189
Eval Time: 23.509796857833862
--epoch: 4300/20000 | ECR: 43.43054 | IS: 15.12631 | WIS: 71.00253 | PDIS: 14.14867 | PDWIS: 3.18752 | DR: 656.33333 --

Eval Time: 23.24097204208374
--epoch: 7800/20000 | ECR: 46.50555 | IS: 30.11683 | WIS: 67.00081 | PDIS: 26.78469 | PDWIS: 5.59874 | DR: 801.92532 --
Pred-> Constructive: 1007, Active: 2570, Passive: 1316, None: 105
Eval Time: 23.101923942565918
--epoch: 7900/20000 | ECR: 46.42586 | IS: 20.67336 | WIS: 74.75028 | PDIS: 18.97644 | PDWIS: 6.21954 | DR: 642.72002 --
Pred-> Constructive: 1167, Active: 2842, Passive: 231, None: 758
Eval Time: 23.274250030517578
--epoch: 8000/20000 | ECR: 47.54500 | IS: 31.98224 | WIS: 76.09333 | PDIS: 29.46989 | PDWIS: 6.58909 | DR: 726.17092 --
Pred-> Constructive: 2707, Active: 1337, Passive: 899, None: 55
Eval Time: 23.235345363616943
--epoch: 8100/20000 | ECR: 47.55012 | IS: 22.00004 | WIS: 76.35829 | PDIS: 20.22640 | PDWIS: 5.52371 | DR: 666.92026 --
Pred-> Constructive: 3470, Active: 121, Passive: 284, None: 1123
Eval Time: 23.29237723350525
--epoch: 8200/20000 | ECR: 48.38805 | IS: 31.27642 | WIS: 63.79736 | PDIS: 28.69280 | PDWIS: 4.85658 | DR: 766.7

Eval Time: 23.12685227394104
--epoch: 11700/20000 | ECR: 54.07612 | IS: 27.18717 | WIS: 73.17528 | PDIS: 25.06094 | PDWIS: 10.07452 | DR: 639.37466 --
Pred-> Constructive: 1909, Active: 958, Passive: 2043, None: 88
Eval Time: 23.193170070648193
--epoch: 11800/20000 | ECR: 52.26047 | IS: 30.61383 | WIS: 72.61581 | PDIS: 28.47739 | PDWIS: 7.85819 | DR: 684.92038 --
Pred-> Constructive: 3071, Active: 437, Passive: 1461, None: 29
Eval Time: 23.16294813156128
--epoch: 11900/20000 | ECR: 51.18113 | IS: 28.51315 | WIS: 77.27807 | PDIS: 26.34628 | PDWIS: 8.79722 | DR: 656.78468 --
Pred-> Constructive: 2187, Active: 867, Passive: 1446, None: 498
Eval Time: 23.16779112815857
--epoch: 12000/20000 | ECR: 54.31620 | IS: 33.59469 | WIS: 81.42142 | PDIS: 30.88688 | PDWIS: 7.87117 | DR: 723.72724 --
Pred-> Constructive: 2990, Active: 824, Passive: 293, None: 891
Eval Time: 23.11808204650879
--epoch: 12100/20000 | ECR: 56.15858 | IS: 36.64736 | WIS: 80.89412 | PDIS: 33.12160 | PDWIS: 13.47617 | DR: 679

Eval Time: 23.090989112854004
--epoch: 15600/20000 | ECR: 50.98170 | IS: 22.95046 | WIS: 71.38920 | PDIS: 21.15869 | PDWIS: 8.72598 | DR: 572.88458 --
Pred-> Constructive: 1998, Active: 1676, Passive: 1272, None: 52
Eval Time: 23.029202938079834
--epoch: 15700/20000 | ECR: 54.30980 | IS: 23.08245 | WIS: 70.39470 | PDIS: 21.12947 | PDWIS: 8.43606 | DR: 643.96321 --
Pred-> Constructive: 1697, Active: 1809, Passive: 1454, None: 38
Eval Time: 23.106637716293335
--epoch: 15800/20000 | ECR: 55.23373 | IS: 21.80466 | WIS: 61.89505 | PDIS: 20.08919 | PDWIS: 5.35422 | DR: 714.47442 --
Pred-> Constructive: 2219, Active: 2149, Passive: 612, None: 18
Eval Time: 23.024715900421143
--epoch: 15900/20000 | ECR: 55.29870 | IS: 15.09005 | WIS: 70.40820 | PDIS: 14.18457 | PDWIS: 4.02118 | DR: 732.40693 --
Pred-> Constructive: 2832, Active: 986, Passive: 1170, None: 10
Eval Time: 23.091798067092896
--epoch: 16000/20000 | ECR: 57.39984 | IS: 24.60619 | WIS: 72.21524 | PDIS: 22.59617 | PDWIS: 13.01149 | DR:

...

Eval Time: 25.781837224960327
--epoch: 4500/20000 | ECR: 49.86387 | IS: 25.89296 | WIS: 67.59060 | PDIS: 23.80650 | PDWIS: 7.36216 | DR: 689.11646 --
Pred-> Constructive: 2520, Active: 0, Passive: 2478, None: 0
Eval Time: 25.827509880065918
--epoch: 4600/20000 | ECR: 49.30114 | IS: 0.29343 | WIS: 7.14500 | PDIS: 0.29343 | PDWIS: 0.12219 | DR: 558.28377 --
Pred-> Constructive: 0, Active: 0, Passive: 4998, None: 0
Eval Time: 25.829063892364502
--epoch: 4700/20000 | ECR: 49.26715 | IS: 15.06928 | WIS: 67.48264 | PDIS: 14.09163 | PDWIS: 3.18599 | DR: 782.88962 --
Pred-> Constructive: 0, Active: 3323, Passive: 1675, None: 0
Eval Time: 26.32573103904724
--epoch: 4800/20000 | ECR: 47.58361 | IS: 28.43836 | WIS: 67.09816 | PDIS: 24.98083 | PDWIS: 5.94907 | DR: 819.21085 --
Pred-> Constructive: 269, Active: 2791, Passive: 1938, None: 0
Eval Time: 26.164997816085815
--epoch: 4900/20000 | ECR: 46.29178 | IS: 2.31627 | WIS: 34.70846 | PDIS: 2.31627 | PDWIS: 0.91965 | DR: 577.36809 --
Pred-> Constr

Eval Time: 26.695120096206665
--epoch: 8400/20000 | ECR: 50.56981 | IS: 1.05570 | WIS: 20.56500 | PDIS: 1.05570 | PDWIS: 0.43431 | DR: 600.83556 --
Pred-> Constructive: 94, Active: 501, Passive: 4371, None: 32
Eval Time: 26.664772748947144
--epoch: 8500/20000 | ECR: 51.39373 | IS: 16.47642 | WIS: 71.52329 | PDIS: 15.00814 | PDWIS: 4.58174 | DR: 634.66592 --
Pred-> Constructive: 1606, Active: 1149, Passive: 2220, None: 23
Eval Time: 26.661784887313843
--epoch: 8600/20000 | ECR: 51.56498 | IS: 4.23418 | WIS: 45.19548 | PDIS: 4.01073 | PDWIS: 1.28753 | DR: 547.21907 --
Pred-> Constructive: 1187, Active: 606, Passive: 3193, None: 12
Eval Time: 26.477787017822266
--epoch: 8700/20000 | ECR: 50.65223 | IS: 22.06056 | WIS: 70.59380 | PDIS: 20.15313 | PDWIS: 5.72732 | DR: 726.56506 --
Pred-> Constructive: 2183, Active: 1366, Passive: 1407, None: 42
Eval Time: 26.73074698448181
--epoch: 8800/20000 | ECR: 50.77847 | IS: 514.93092 | WIS: 62.58512 | PDIS: 348.01994 | PDWIS: 9.47014 | DR: 1276.11093

Eval Time: 24.971224069595337
--epoch: 12300/20000 | ECR: 60.00105 | IS: 8.57304 | WIS: 57.09496 | PDIS: 8.23959 | PDWIS: 2.08463 | DR: 900.68466 --
Pred-> Constructive: 2328, Active: 956, Passive: 1296, None: 418
Eval Time: 25.16976261138916
--epoch: 12400/20000 | ECR: 63.91206 | IS: 499.18220 | WIS: 62.41865 | PDIS: 333.91047 | PDWIS: 7.36798 | DR: 1239.11954 --
Pred-> Constructive: 2965, Active: 798, Passive: 925, None: 310
Eval Time: 25.01572895050049
--epoch: 12500/20000 | ECR: 60.71610 | IS: 5.25887 | WIS: 47.10011 | PDIS: 4.98729 | PDWIS: 2.83152 | DR: 528.23472 --
Pred-> Constructive: 688, Active: 841, Passive: 2930, None: 539
Eval Time: 27.178532123565674
--epoch: 12600/20000 | ECR: 56.29603 | IS: 6.79929 | WIS: 52.45554 | PDIS: 6.46470 | PDWIS: 2.47384 | DR: 522.90642 --
Pred-> Constructive: 1584, Active: 916, Passive: 2106, None: 392
Eval Time: 27.239524126052856
--epoch: 12700/20000 | ECR: 59.12006 | IS: 52.47675 | WIS: 81.13071 | PDIS: 46.79729 | PDWIS: 10.92997 | DR: 1012

Eval Time: 31.35903525352478
--epoch: 16100/20000 | ECR: 58.29347 | IS: 17.88467 | WIS: 64.36829 | PDIS: 16.36445 | PDWIS: 7.35370 | DR: 668.08076 --
Pred-> Constructive: 2264, Active: 603, Passive: 1968, None: 163
Eval Time: 26.019238710403442
--epoch: 16200/20000 | ECR: 55.31383 | IS: 63.11536 | WIS: 79.05081 | PDIS: 52.03147 | PDWIS: 11.74327 | DR: 749.66597 --
Pred-> Constructive: 2386, Active: 887, Passive: 1530, None: 195
Eval Time: 25.817668914794922
--epoch: 16300/20000 | ECR: 58.08501 | IS: 18.57217 | WIS: 64.46073 | PDIS: 17.04966 | PDWIS: 5.49099 | DR: 709.65621 --
Pred-> Constructive: 3188, Active: 742, Passive: 938, None: 130
Eval Time: 26.06474804878235
--epoch: 16400/20000 | ECR: 58.02396 | IS: 17.08951 | WIS: 68.99560 | PDIS: 16.31721 | PDWIS: 4.78955 | DR: 599.89500 --
Pred-> Constructive: 1096, Active: 1795, Passive: 2047, None: 60
Eval Time: 26.00522208213806
--epoch: 16500/20000 | ECR: 56.65546 | IS: 39.74740 | WIS: 79.05853 | PDIS: 34.76216 | PDWIS: 14.82089 | DR: 

Eval Time: 25.753206968307495
--epoch: 19900/20000 | ECR: 54.95991 | IS: 27.77121 | WIS: 74.36194 | PDIS: 24.99870 | PDWIS: 7.39545 | DR: 737.39744 --
Pred-> Constructive: 3271, Active: 626, Passive: 1072, None: 29
Eval Time: 25.691134929656982
--epoch: 20000/20000 | ECR: 58.66978 | IS: 47.88999 | WIS: 80.98942 | PDIS: 42.15344 | PDWIS: 10.91479 | DR: 772.19952 --
Pred-> Constructive: 2466, Active: 1533, Passive: 994, None: 5
True-> Constructive: 2009, Active: 2074, Passive: 445, None: 470
Pred-> Constructive: 2466, Active: 1533, Passive: 994, None: 5
-> True Reward: 54.63407/30.52030, Pred Reward: 58.66978/18.42337


In [5]:
# target = 'ECR'
# step = 1

# y = []
# for i in range(int(len(eval_df)/step)):
#     y.append(eval_df.loc[i*step:i*step+step, target].mean())

# plt.plot(range(len(y)), y)

# OTHER EXPERIMENTAL DETAILS

In [1]:
import pandas as pd
df = pd.read_pickle('../temp/df_all_norm_cluster.pkl')
df.groupby(['action']).count()

df['delayed_reward'] = pd.to_numeric(df['delayed_reward'])
df.groupby(['episode_id']).sum()[['delayed_reward']].describe()

,delayed_reward
count,487.000000
mean,54.634066
std,30.551682
min,-100.000000
25%,36.360000
50%,60.000000
75%,77.780000
max,100.000000


In [16]:
df.loc[df['done']==True].describe()

,cluster,ns_cluster,action_proba,ns_act_0_proba,ns_act_1_proba,ns_act_2_proba,ns_act_3_proba
count,487.000000,487.0,487.000000,4.870000e+02,4.870000e+02,4.870000e+02,4.870000e+02
mean,2.088296,1.0,0.362423,9.000000e-02,9.000000e-02,4.200000e-01,3.900000e-01
std,1.512093,0.0,0.114959,8.335235e-16,8.335235e-16,3.723072e-15,4.112049e-15
min,0.000000,1.0,0.080000,9.000000e-02,9.000000e-02,4.200000e-01,3.900000e-01
25%,1.000000,1.0,0.390000,9.000000e-02,9.000000e-02,4.200000e-01,3.900000e-01
50%,3.000000,1.0,0.400000,9.000000e-02,9.000000e-02,4.200000e-01,3.900000e-01
75%,3.000000,1.0,0.420000,9.000000e-02,9.000000e-02,4.200000e-01,3.900000e-01
max,4.000000,1.0,0.440000,9.000000e-02,9.000000e-02,4.200000e-01,3.900000e-01


In [17]:
df['delayed_reward'].unique()

array([0, 30.0, 88.89, 85.71, 71.43, 62.5, 66.67, 45.45, 40.0, 42.86,
       100.0, 44.44, -12.5, 83.33, 25.0, 72.73, 37.5, 57.14, 20.0, 28.57,
       75.0, 33.33, 27.27, 22.22, 50.0, 80.0, 77.78, 87.5, 55.56, 60.0,
       10.0, 12.5, 70.0, -66.67, 9.09, 90.0, 18.18, 81.82, 63.64, -22.22,
       11.11, 54.55, 14.29, -9.09, -100.0, -50.0, -33.33, 41.67, 16.67,
       36.36, -25.0], dtype=object)